# Topic Modeling using BERT
In this activity we will use BERTopic, which is a topic modeling technique that leverages BERT embeddings and a class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.

For more details on BERTopic, see:
https://maartengr.github.io/BERTopic/index.html

https://github.com/MaartenGr/BERTopic

You can compare the resulting topics from this activity with topics we derived in our earlier activity using LDA for topic modeling.

In [ ]:
#run once!
!pip install bertopic
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 58.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     

## Data & Scenario

We will use the same dataset on restaurant reviews we used for the earlier activity using LDA for topic modeling so that you can compare the results.  

As explained earlier, we can explore whether there are certain topics that people write about in their reviews. These topics can be used to come up with different strategies to engage users on online platforms or other channels.  

This is a small dataset for learning purposes and to avoid long processing times.
You can use any other textual data as input. Depending on the data format, you may have to use different functions to import your text data. Once you have your data imported as a dataframe, where one colum contains the *documents*, the rest will be the same.

Download the file "**Restaurant_Reviews.tsv**" form elearn and upload it to your session before processing.

In [ ]:
# importing restaurant reviews dataset
import pandas as pd
df=pd.read_csv('Restaurant_Reviews.tsv',delimiter="\t")
df.head(2)

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0


The sentiment for each review has been manually labeled for this dataset, we will not be using it for this activity; we only use the Review content.

There are 996 unique reviews (documents, in NLP terminology).

In [ ]:
len(df)
docs=df['Review']
docs=docs.drop_duplicates() #drop duplicate reviews
docs=docs.values
len(docs)

996

In [ ]:
#let's take a look at a document
docs[2]

'Not tasty and the texture was just nasty.'

## Pre-processing
Here we won't need to use the pre-processing steps as we did in our earlier activity, since the library we are using is going to automatically apply the needed text pre-processing.

## Importing BERTopic

In [ ]:
#import BERTTopic
from bertopic import BERTopic

This package is very easy to use, but there are several steps happening in the background (which are customizable, btw) and utilizes advanced pre-trained models (more on this later).

## Embeddings
Let's recall our previous activities when we used RNNs for sentiment analysis and document classification.
Remember those RNN models had an embedding layer (the first layer) that would convert tokens (words represented as integers) to vectors (word vectors); in sum, it would create word embeddings (vectors) from tokens so that similar words would be close in the resulting vector-space.

Here, we have a similar process at the sentence-level (instead of token-level). The first step (happens in the background) is to create sentence embeddings using pre-trained models using what is known as "sentence-transformers"; sentence-transformers convert sentences into vector representations. These models are usually trained on very large collections of text (some for multilple languages) where the training goal is for the model to be able to predict some missing part of text (e.g., at the word or sentence level). This process can also be done at the document level, i.e., document embeddings.

Here is a repository for pre-trained models https://huggingface.co/models. Depending on what sort of textual data you are working with (e.g., scientific articles, social media, etc.), you might want to use a library that is trained on type of text that is similar in nature to what you have.

We will be using the default "distilbert-base-nli-mean-tokens" model (link to original paper https://arxiv.org/abs/1910.01108 ) for the english language.

In [ ]:
# creating a instance of bertopic for "english", we are setting the parameters to save topic probablities (will use them later for visualization)
model = BERTopic(language="english",calculate_probabilities=True,verbose=True)


Notice that we are not specifying the number of topics here, contrary to when we do topic modeling using LDA.

You could think of this as sth similar to what we had when we used DBSCAN for clustering where we don't specifiy number of clusters (vs. k-means where we had to specify the number of clusters).

## Running BERTopic on our corpus

In [ ]:
# apply the model on our documents and save both topics and probabilities (probability of each document belonging to any of the topics)
topics, probabilities = model.fit_transform(docs)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2023-04-13 08:21:01,495 - BERTopic - Transformed documents to Embeddings
2023-04-13 08:21:14,050 - BERTopic - Reduced dimensionality
2023-04-13 08:21:14,164 - BERTopic - Clustered reduced embeddings


## Derived Topics and their frequency

Topic **-1** refers to all documents that did not have any topics assigned (outlier topic).

In [ ]:
# number of topics and freq (number of documents assigned to each topic)
model.get_topic_freq().shape

(25, 2)

In [ ]:
# Topic -1 refers to all documents that did not have any topics assigned (outlier topic).
model.get_topic_freq()

,Topic,Count
0,-1,335
1,0,61
2,1,56
3,2,53
4,3,47
5,4,43
6,5,43
7,6,42
8,7,39
9,8,32


### Question 1
How many topics were derived (aside from the outlier cluster)?

Answer: 24 topics.

###Question 2
How many documents are assigned to the first and second topic?

Answer: 56 documents to topic 1.
53 documents to topic 2.

## Top words for a topic
Let's take a look at top words that represent the first topic (topic=1), which has the most documents. You can simply change the topic number to look at the top words for other topics.

In [ ]:
# get top words for first topic
model.get_topic(1)

[('experience', 0.08575303234845513),
 ('disappointed', 0.058081079622886514),
 ('was', 0.05694773405343657),
 ('it', 0.052638412876519575),
 ('good', 0.04150761584285264),
 ('very', 0.040579582117416794),
 ('you', 0.03835505836922988),
 ('ambiance', 0.038079280180839954),
 ('not', 0.03772586668890369),
 ('overall', 0.036857514510977525)]

## Visualizing topics
We can also visualize the derived topics (note that we installed the visualiazation library in the beginning of this notebook).

In [ ]:
model.visualize_topics()

## Document-Topic probablities

Each document in our corpus has a probability for belonging to each of the derived topics; we can derive these probabilities to use them for some other task (for example, as features for some predictive modeling task).


In [ ]:
probabilities.shape

(996, 24)

In [ ]:
# for example, we can export all these document-topic probabilities as a csv file.
# pd.DataFrame(probabilities).to_csv("probs.csv")

## Visualize Topic probability distribution
We can also visualize the Topic probability distribution for a specific document.
Note that topics with a probablity beloew the specified threshold are not shown.

In [ ]:
docs[10] # review number 5

'Service was very prompt.'

In [ ]:
# probablity of doc[4] (review 5) belonging to each of the topics
model.visualize_distribution(probabilities[10],min_probability=0.005)

### Question 3
Which topic does the 10th review belong to?

Answer: Topic 4.

## What topic would a new review be most similar to?
Let's see which topic(s) a new review would be more similar to.

We have the option to derive similarity of a new piece of text to the derived topcis (in terms of cosine similarity between embeddings).

In [ ]:
new_review="The food was too salty but I liked the atmosphere."

In [ ]:
model.find_topics(new_review)

([8, 6, 16, 19, 20],
 [0.32735023533094737,
  0.3121043308709059,
  0.3071931635621252,
  0.270192204328968,
  0.2601766192034405])

###Question 4
Which topic(s) is the new review most simlar to?

Answer: Topics 8, 6, 16, 19, 20.

In [ ]:
model.get_topic(15)

[('eat', 0.09061098662097605),
 ('here', 0.07391281197873827),
 ('eating', 0.07128795475715385),
 ('again', 0.06732536520354557),
 ('there', 0.059133167922716896),
 ('enjoyed', 0.05138186753251858),
 ('each', 0.04928983477945591),
 ('twice', 0.04928983477945591),
 ('would', 0.04855981969345222),
 ('be', 0.04594797935360452)]